In [3]:
import dlib
import scipy.misc
import numpy as np
import os
import pandas as pd
face_detector = dlib.get_frontal_face_detector()
shape_predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')
face_recognition_model = dlib.face_recognition_model_v1('dlib_face_recognition_resnet_model_v1.dat')
TOLERANCE = 0.5
df = pd.read_csv('record.csv')
df

,F_ID,Name,Charges,Location
0,1,Sreekiran A R,Un ethical Hacker,Bangalore
1,2,Vivek Kumar,Blackmail,Jaipur
2,3,Arpan Naik,Flight Hijacking,Rajasthan
3,4,Runcy,Black Money Dealer,Mars


In [4]:
def get_face_encodings(path_to_image):
    image = scipy.misc.imread(path_to_image)
    # Detect faces using the face detector
    detected_faces = face_detector(image, 1)
    shapes_faces = [shape_predictor(image, face) for face in detected_faces]
    # For every face detected, compute the face encodings
    return [np.array(face_recognition_model.compute_face_descriptor(image, face_pose, 1)) for face_pose in shapes_faces]

In [5]:
def compare_face_encodings(known_faces, face):
    return (np.linalg.norm(known_faces - face, axis=1) <= TOLERANCE)

In [6]:
def find_match(known_faces, names, face):
    matches = compare_face_encodings(known_faces, face)
    count = 0
    for match in matches:
        if match:
            return names[count]
        count += 1
    return 'Not Found'

In [7]:
def details(name):
    return df[df['Name'] == name]

In [15]:
image_filenames = filter(lambda x: x.endswith('.jpg'), os.listdir('images/'))
image_filenames = sorted(image_filenames)
paths_to_images = ['images/' + x for x in image_filenames]
face_encodings = []
for path_to_image in paths_to_images:
    face_encodings_in_image = get_face_encodings(path_to_image)
    if len(face_encodings_in_image) != 1:
        print("Please change image: " + path_to_image + " - it has " + str(len(face_encodings_in_image)) + " faces; it can only have one")
        exit()
    face_encodings.append(get_face_encodings(path_to_image)[0])

In [ ]:
import cv2
import sys
import numpy as np
import time
from collections import Counter
faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
from kafka import KafkaConsumer,KafkaProducer
consumer = KafkaConsumer('detection',bootstrap_servers =['172.26.42.131:9092'])
#consumer2 = KafkaConsumer('motionsensor',bootstrap_servers = ['Extrack.bridgei2i.in:9092'])
try:
    consumer.poll()
    consumer.seek_to_end()
except:
    print "latest"
producer =  KafkaProducer(bootstrap_servers='172.26.42.131:9092')
found = []
head_count = []
p = time.time()

while True:
    msg= next(consumer)
    nparr = np.fromstring(msg.value, np.uint8)
    flags = cv2.IMREAD_COLOR
    frame = cv2.imdecode(nparr,flags)
    #converting to Grayscale
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    #Detecting the faces
    faces = faceCascade.detectMultiScale(
        gray,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(30, 30),
        flags=cv2.CASCADE_SCALE_IMAGE
    )
    #print len(faces)
    head_count.append(len(faces))
    #matching each face with known faces
    for (x,y,w,h) in faces:
        test = frame[y-int(.21*y):y+h+int(.21*y),x-int(.13*x):x+h+int(.13*x),:]
        cv2.imwrite('test.jpg',test)
        cv2.imwrite('new.jpg',frame)
        names = [x[:-4] for x in image_filenames]

        try:    
            face_encodings_in_image = get_face_encodings("test.jpg")
            #print type(face_encodings_image[0]),names,tye(names),type(names[0]),face_encodings_image[0].shape
            match = find_match(face_encodings, names, face_encodings_in_image[0])
            if match:
                found.append(match) 
                if match != 'Not Found':
                    #print match
                    cv2.imwrite('criminals/'+str(match)+'.jpg',test)

            else:
                print "Not Found"
        except:
            continue

        q = time.time()
        if q-p >2:
            out = []
            found = filter(lambda x:x !='Not Found',found)
            data = Counter(found)
            output = map(lambda (x,y):x ,data.most_common(max(head_count)))
            for name in output:

                sss = details(name)
                l = str([sss[key].values[0] for key in sss.keys()])
                out.append(l)
            producer.send('faceid',str(out)+' Head_count = '+str(max(head_count)))
            p = time.time()
            head_count = []


latest


In [ ]:
l = []
for key in sss.keys():
    l.append(sss[key].values[0])
str(l)
pss = str([sss[key].values[0] for key in sss.keys()])


In [ ]:
str(sss)

In [ ]:
b = a.split('\\\n0')
c = str(b[1]).split('\n\n')
d = str(b[1]).split('\n\n')[0]
e = d+((str(c[1])).split('\n0')[1])
e

In [ ]:
a = str(details(name))
                        b = str(details(name)).split('\\\n')
                        c = str(b[1]).split('\n\n')
                        d = str(b[1]).split('\n\n')[0]
                        e = d+((str(c[1])).split('\n')[1])
                        out.append(filter(lambda x: '\n' not in x and len(x)>1 ,e.split('  ')))